In [44]:
import arcpy;
import os,sys;
import requests,csv,codecs;
import sqlite3;

if not os.path.exists('scratch'):
    os.mkdir('scratch');
    
if not os.path.exists('scratch' + os.sep + 'working.gpkg'):
    arcpy.management.CreateSQLiteDatabase(
         out_database_name = os.path.abspath('scratch' + os.sep + 'working')
        ,spatial_type      = 'GEOPACKAGE_1.3'
    );


### Download the WQP station dataset
I don't know your criteria so I just downloaded stations in Rhode Island for a small set

In [45]:
target = 'scratch' + os.sep + 'wqp_input.csv';

if os.path.exists(target):
    os.remove(target);

row_cnt = 0;
with open(target,'w',newline='',encoding='utf-8') as f:
    writer = csv.writer(f,delimiter=',');
    
    parms = {'statecode':'US:44','mimeType':'csv','sorted':'no'};
    r = requests.get('https://www.waterqualitydata.us/data/Station/search',params=parms,stream=True);
    rd = [line.decode('utf-8') for line in r.iter_lines()];
    cr = csv.reader(rd);
    
    header = True;
    for row in cr:
        
        if header:
            header = False;

            for idx,ch in enumerate(row):
                row[idx] = row[idx].replace('/','_');

            writer.writerow(row);

        else:
            for idx,ch in enumerate(row):
                row[idx] = row[idx].replace(u'\u00b6','');

            writer.writerow(row);
            row_cnt += 1;
    
print("Downloaded " + str(row_cnt) + " records from WQP.");

Downloaded 5135 records from WQP.


### Load WQP CSV into Geopackage

In [46]:
source = 'scratch' + os.sep + 'wqp_input.csv';
target = 'scratch' + os.sep + 'working.gpkg' + os.sep + 'wqp_input';

if arcpy.Exists(target):
    arcpy.Delete_management(target);
    
arcpy.conversion.TableToTable(
     in_rows   = source
    ,out_path  = os.path.dirname(target)
    ,out_name  = os.path.basename(target)
);

arcpy.management.AddIndex(
     in_table   = target
    ,fields     = 'MonitoringLocationIdentifier'
    ,index_name = 'MonitoringLocationIdentifier_idx'
);

gpkg_cnt = arcpy.GetCount_management(target)[0];
print("Loaded " + str(gpkg_cnt) + " WQP records into Geopackage.");

Loaded 5135 WQP records into Geopackage.


### Download Office of WATER Linked Data for WQP


In [47]:
target = 'scratch' + os.sep + 'working.gpkg' + os.sep + 'owld_input';

if arcpy.Exists(target):
    arcpy.Delete_management(target);
    
host         = "watersgeo.epa.gov";
service_path = "/arcgis/rest/services/owld/wqp/MapServer/3";
#where = "1=1";
where = "geogstate = 'RI'";

headers = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain"};
parms   = {"f": "json"};
r = requests.post('https://' + host + service_path,params=parms);
r_json = r.json();
extraction_amount = r_json['maxRecordCount'];

if not 'currentVersion' in r_json:
    raise ValueError("Error, unable to query https://" + host + service_path);
    
parms = {"where": where,"returnCountOnly": "true","f": "json"};
r = requests.post('https://' + host + service_path + '/query',params=parms);
r_json = r.json();
total_records = r_json['count'];

result_offset = 0;
initial_hit = True;
while result_offset <= total_records:
 
    parms = {
         "where": where
        ,"outFields": "*"
        ,"resultOffset": result_offset
        ,"resultRecordCount": extraction_amount
        ,"returnGeometry": "true"
        ,"f": "json"
    };

    r = requests.post('https://' + host + service_path + '/query',params=parms);
    json_data = r.json();
    ef = arcpy.AsShape(json_data,True)
   
    if initial_hit:
        arcpy.management.CopyFeatures(ef,target)
        initial_hit = False;
    else:
        arcpy.Append_management(ef,target,"NO_TEST");
      
    result_offset += extraction_amount;

arcpy.management.AddIndex(
     in_table   = target
    ,fields     = 'source_featureid'
    ,index_name = 'source_featureid_idx'
);

owld_cnt = arcpy.GetCount_management(target)[0];
print("Downloaded " + str(owld_cnt) + " OWLD records into Geopackage.");

Downloaded 4898 OWLD records into Geopackage.


### Join WQP and OWLD together to create new table with NHDPlus reach measures

Note I am somewhat avoiding the issue of what to do about WQP stations newly added since the last WQP refresh of OWLD. 

In [54]:
target  = 'scratch' + os.sep + 'working.gpkg' + os.sep + 'wqp_indexed';
source  = 'scratch' + os.sep + 'working.gpkg' + os.sep + 'wqp_input';
source2 = 'scratch' + os.sep + 'working.gpkg' + os.sep + 'owld_input';

if arcpy.Exists(target):
    arcpy.Delete_management(target);

arcpy.conversion.TableToTable(
     in_rows      = source
    ,out_path     = os.path.dirname(target)
    ,out_name     = os.path.basename(target)
);

arcpy.management.JoinField(
     in_data    = target
    ,in_field   = 'MonitoringLocationIdentifier'
    ,join_table = source2
    ,join_field = 'source_featureid'
    ,fields     = ['feature_permanent_identifier','reachcode','measure','navigable']
);

arcpy.management.AddField(
     in_table   = target
    ,field_name = 'nhdplusid'
    ,field_type = 'FLOAT'
);

with arcpy.da.UpdateCursor(
     in_table    = target
    ,field_names = ['feature_permanent_identifier','nhdplusid']
) as cursor:
    for row in cursor:
        if row[0] is None or row[0] == '':
            cursor.deleteRow();
        else:
            row[1] = float(row[0]);
            cursor.updateRow(row);
            
arcpy.management.DeleteField(
     in_table   = target
    ,drop_field = 'feature_permanent_identifier'
);

join_cnt = arcpy.GetCount_management(target)[0];
print("Joined together " + str(join_cnt) + " records.");

Joined together 4896 records.
